In [1]:
import pandas as pd
import nltk 
from nltk.stem import WordNetLemmatizer
from gensim.models import Word2Vec
import pickle      

In [3]:
df_data = pickle.load(open('5.PM_MANMOHAN_Speech_Cleaned_string.pickle', 'rb'))

In [5]:
df_data

,index,Date,Location,Title,Speech,Cleaned_Speech,Cleaned_Speech_string
0,MSS1,2004-06-24,New Delhi,Prime Minister 's Address to the Nation,"My Fellow Citizens,I speak to you with a deep...",['fellow citizen speak deep sense humility ful...,fellow citizen speak deep sense humility fully...
1,MSS2,2004-06-29,New Delhi,PM's inaugural address at the Chief Ministers'...,"""I welcome you all to this Conference which w...",['welcome conference take stock effort rural d...,welcome conference take stock effort rural dev...
2,MSS3,2004-07-03,New Delhi,"PM's inaugural address at the ""Dialogue on Min...","""It gives me great pleasure to be amongst you...","['give great pleasure amongst morning', 'parti...",give great pleasure amongst morning particular...
3,MSS4,2004-07-26,New Delhi,PM's speech at the CSIR Society Meeting,"Distinguished Members of the CSIR Society, La...",['distinguished member csir society lady gentl...,distinguished member csir society lady gentlem...
4,MSS5,2004-07-31,Bangkok,PM’ statement at the Press Conference,"\n\t""My two days in Bangkok have been very use...","['two day bangkok useful productive', 'first b...",two day bangkok useful productive first bimste...
...,...,...,...,...,...,...,...
1346,MSS1397,2014-02-21,New Delhi,PM’s farewell speech to the 15th Lok Sabha,"""Madam Speaker,\n\nAs we come to the end of th...",['madam speaker come end journey fifteenth lok...,madam speaker come end journey fifteenth lok s...
1347,MSS1398,2014-02-26,New Delhi,PM's speech at the foundation laying ceremony ...,\n\t“Today marks a very important step forward...,['today mark important step forward towards es...,today mark important step forward towards esta...
1348,MSS1399,2014-03-01,New Delhi,PM’s address at the National Awards Ceremony f...,\n\t“I am very happy to participate in the Nat...,['happy participate national award function mi...,happy participate national award function mini...
1349,MSS1400,2014-03-04,"Nay Pyi Taw, Myanmar",PM’s statement at 3rd BIMSTEC Summit,\n\t“I am delighted to return to this beautifu...,['delighted return beautiful city nay pyi taw ...,delighted return beautiful city nay pyi taw th...


In [6]:
def model(corpus):
    corpus_tokenized = [nltk.word_tokenize(sentence) for sentence in corpus]
    model = Word2Vec(sentences=corpus_tokenized, vector_size=300, window=5, min_count=1, workers=4)
    return model 


In [7]:
# Defining the function for vectorization 
def predict(model, speech):
    vectors = []

    for sentence in speech:
        sent_vec = []
        for word in sentence:
            if word in model.wv.key_to_index:
                # If the word is in the Word2Vec model's vocabulary, extract its vector
                word_vector = model.wv.get_vector(word)
                sent_vec.append(word_vector)
        vectors.append(sent_vec)

    return vectors 



In [8]:
# Defining a function to count the number of unique words in each sentences 
def count_unique_words(j):
    m = j.replace('[','').replace(']','').replace("'","").split(', ')
    l = []
    for k in m:
        l += k.split(" ")
    length = len(l)
    return length 

In [9]:
# Creating the corpus of all the speeches to train the model 
corpus_Manmohan = []
for i in df_data['Cleaned_Speech']:
    corpus_Manmohan += i.replace('[','').replace(']','').replace("'","").split(', ')

In [10]:
empty_count = 0
for element in corpus_Manmohan:
    if element.strip() == '':
        empty_count += 1

print(f'Total empty elements in the list: {empty_count}')

Total empty elements in the list: 56


In [11]:
len(corpus_Manmohan)

76646

In [12]:
corpus_Manmohan = [element for element in corpus_Manmohan if element.strip() != '']

In [13]:
len(corpus_Manmohan)

76590

In [14]:
# Training the model 
model_manmohan = model(corpus_Manmohan)

In [15]:
Word2vec_model_MM = open("Word2vec_model_MM.pickle", "wb")
pickle.dump(model_manmohan, Word2vec_model_MM)
Word2vec_model_MM.close()

In [15]:
# Creating the list containing each of the speeches 
speech_manmohan = []
for i in df_data['Cleaned_Speech']:
    speech_manmohan.append(i.replace('[','').replace(']','').replace("'","").split(', '))

In [16]:
# Vectorize each speech 
speech_manmohan_vector = []
for sp in speech_manmohan:
    speech_manmohan_vector.append(predict(model_manmohan, sp)) 

In [1]:
speech_manmohan_vector[0][0]

NameError: name 'speech_manmohan_vector' is not defined

In [20]:
import pandas as pd
import numpy as np

# initialize an empty list to store the row dictionaries
rows = []

# loop over all speeches, sentences, and words
for speech_number, speech in enumerate(speech_manmohan_vector):
    for sentence_number, sentence in enumerate(speech):
        # initialize a vector of zeros with 300 dimensions
        sentence_vector = np.zeros(300)
        for word in sentence:
            # add the ith element of the word vector to the corresponding dimension of the sentence vector
            sentence_vector += np.array(word)
        # divide the sentence vector by the number of words to get the average
        sentence_vector /= len(sentence)
        # create a dictionary with the speech number, sentence number, and sentence vector
        row_dict = {"speech_number": speech_number, "sentence_number": sentence_number}
        for i, dim in enumerate(sentence_vector):
            row_dict["dim_" + str(i)] = dim
        # add the dictionary to the list of rows
        rows.append(row_dict)

# create a dataframe from the list of rows
df = pd.DataFrame(rows)


C:\Users\abcd\AppData\Local\Temp\ipykernel_17904\926604485.py:16: RuntimeWarning: invalid value encountered in divide
  sentence_vector /= len(sentence)


In [21]:
df

,speech_number,sentence_number,dim_0,dim_1,dim_2,dim_3,dim_4,dim_5,dim_6,dim_7,...,dim_290,dim_291,dim_292,dim_293,dim_294,dim_295,dim_296,dim_297,dim_298,dim_299
0,0,0,-0.101121,-0.133594,0.105327,0.005110,0.114845,-0.183630,-0.038635,0.334679,...,-0.108321,0.184278,0.142120,-0.023458,0.111865,0.166980,-0.018790,-0.031468,0.218024,0.112716
1,0,1,-0.126894,-0.245685,0.160768,-0.053816,0.191161,-0.250372,-0.025536,0.458294,...,-0.134293,0.228498,0.181236,-0.036362,0.185661,0.200312,-0.045691,0.052271,0.291911,0.187914
2,0,2,-0.195095,-0.276168,0.204625,-0.005804,0.210510,-0.241672,-0.015830,0.478631,...,-0.214137,0.236851,0.123761,-0.102380,0.313339,0.142880,-0.030740,0.023629,0.314851,0.292724
3,0,3,-0.135498,-0.270984,0.172546,-0.071836,0.204578,-0.273844,-0.022536,0.489191,...,-0.129375,0.250996,0.197854,-0.037117,0.188167,0.231378,-0.051103,0.058713,0.322038,0.199909
4,0,4,-0.116612,-0.167724,0.131053,-0.018342,0.124985,-0.189369,-0.009226,0.377953,...,-0.106487,0.205483,0.143352,-0.034508,0.133707,0.186484,-0.023359,-0.000296,0.253948,0.154519
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76736,1400,18,-0.049997,-0.135540,0.103790,-0.087589,0.106913,-0.179681,-0.015835,0.380353,...,-0.008932,0.184050,0.180692,0.032714,0.006503,0.259544,-0.044093,0.043620,0.254773,0.073014
76737,1400,19,-0.145479,-0.240693,0.173631,-0.029363,0.183263,-0.228004,-0.013844,0.456541,...,-0.153881,0.231589,0.158915,-0.049336,0.206550,0.175209,-0.032346,0.043693,0.297595,0.219920
76738,1400,20,-0.195351,-0.195300,0.294255,-0.134085,0.199162,-0.159376,-0.024304,0.675626,...,-0.053603,0.215365,0.133658,-0.028614,0.114655,0.324382,-0.030530,0.132647,0.462956,0.313792
76739,1400,21,-0.074820,-0.058425,0.109703,-0.033387,0.079620,-0.104136,0.015927,0.312014,...,-0.018685,0.116231,0.077310,-0.011715,0.058460,0.145577,-0.012314,0.042048,0.188386,0.116541


In [22]:
# group the rows by speech number and compute the mean of each group
df_speech = df.groupby("speech_number").mean()

# reset the index of the dataframe and rename the columns
df_speech = df_speech.reset_index().rename(columns={"index": "speech_number"})

# re-order the columns of the dataframe
df_speech = df_speech[["speech_number"] + ["dim_" + str(i) for i in range(300)]]


In [23]:
df_speech.insert(0, "index", df_data["index"])
df_speech.insert(1, "Date", df_data["Date"])

In [24]:
df_speech['speech_number'] = df_speech['speech_number'] +1

In [25]:
df_speech

,index,Date,speech_number,dim_0,dim_1,dim_2,dim_3,dim_4,dim_5,dim_6,...,dim_290,dim_291,dim_292,dim_293,dim_294,dim_295,dim_296,dim_297,dim_298,dim_299
0,MSS1,2004-06-24,1,-0.133635,-0.208698,0.153252,-0.018163,0.169679,-0.228218,-0.024964,...,-0.141725,0.219170,0.156905,-0.045202,0.190684,0.174107,-0.030890,0.018954,0.272331,0.186929
1,MSS2,2004-06-29,2,-0.132128,-0.205244,0.151674,-0.016370,0.168722,-0.228107,-0.025692,...,-0.141351,0.218654,0.157484,-0.043842,0.188954,0.173623,-0.030622,0.017778,0.270408,0.184131
2,MSS3,2004-07-03,3,-0.121136,-0.187154,0.136084,-0.007777,0.155258,-0.222267,-0.029237,...,-0.133672,0.212247,0.157754,-0.037250,0.170556,0.170506,-0.028744,0.003487,0.254190,0.161891
3,MSS4,2004-07-26,4,-0.124542,-0.205915,0.148893,-0.025025,0.173143,-0.247808,-0.012709,...,-0.126609,0.226469,0.167205,-0.041324,0.193240,0.172307,-0.033045,0.033229,0.262817,0.175516
4,MSS5,2004-07-31,5,-0.119762,-0.230225,0.151355,-0.051486,0.205815,-0.310005,0.007787,...,-0.117269,0.257454,0.200506,-0.039243,0.222123,0.178241,-0.033558,0.054209,0.266589,0.164262
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1396,MSS1397,2014-02-21,1397,-0.128307,-0.200513,0.146014,-0.016739,0.168134,-0.230848,-0.031295,...,-0.135828,0.216066,0.160192,-0.039923,0.179670,0.180175,-0.032079,0.015094,0.268841,0.174228
1397,MSS1398,2014-02-26,1398,-0.139489,-0.216732,0.161994,-0.027038,0.181784,-0.246534,-0.015700,...,-0.139678,0.229431,0.160144,-0.051759,0.209794,0.174897,-0.029514,0.029507,0.278915,0.195888
1398,MSS1399,2014-03-01,1399,-0.141361,-0.216530,0.162765,-0.019296,0.179809,-0.240138,-0.019059,...,-0.148955,0.228221,0.159166,-0.052323,0.211110,0.170451,-0.029574,0.026169,0.278660,0.199930
1399,MSS1400,2014-03-04,1400,-0.132841,-0.222096,0.154925,-0.025769,0.181673,-0.242872,-0.024772,...,-0.141557,0.223231,0.163967,-0.044911,0.199782,0.176971,-0.035357,0.030568,0.275875,0.188822


In [26]:
# Saving the speech in pickle format 
speech_MM_vectors = open("speech_MM_vectors.pickle", "wb")
pickle.dump(df_speech, speech_MM_vectors)
speech_MM_vectors.close()